# 数据聚合和分组操作

对数据集进行分类并将功能应用于每个组（无论是聚合还是转换），通常是数据分析工作流的关键组成部分。加载，合并和准备数据集后，您可能需要计算组统计信息或可能地透视表以用于报告或可视化目的。pandas提供了灵活的groupby界面，使您能够以自然的方式对数据集进行切片，切块和汇总。

关系数据库和SQL（代表“结构化查询语言”）流行的原因之一是数据的连接，过滤，转换和聚合的简便性。但是，诸如SQL之类的查询语言在某种程度上可以执行的组操作有所限制。如您所见，借助Python和pandas的表现力，我们可以利用任何接受pandas对象或NumPy数组的函数来执行相当复杂的组操作。在本章中，您将学习如何：

- 使用一个或多个键（以函数，数组或DataFrame列名的形式）将pandas对象拆分为多个部分
- 计算组摘要统计信息，例如计数，均值或标准差，或用户定义的函数
- 应用组内转换或其他操作，例如归一化，线性回归，等级或子集选择
- 计算数据透视表和交叉表
- 执行分位数分析和其他统计组分析

> 时间序列数据的聚合是groupby的一种特殊用例，在本书中称为重采样，第11章将对此进行单独处理。

## 10.1 GroupBy 机制

Hadley Wickham是R编程语言的许多流行软件包的作者，创造了术语split-apply-combine来描述组操作。在该过程的第一阶段，根据提供的一个或多个键，将熊猫对象中包含的数据（无论是系列，数据框还是其他）分为几组。分割是在对象的特定轴上执行的。例如，一个DataFrame可以按其行（axis = 0）或其列（axis = 1）分组。完成此操作后，会将一个函数应用于每个组，从而产生一个新值。最后，所有这些功能应用程序的结果都组合成一个结果对象。最终对象的形式通常取决于对数据执行的操作。有关简单组聚合的模型，请参见图10-1。

![10-1.bmp](images/10-1.bmp)

每个分组键可以采用多种形式，并且键不必都具有相同的类型：

- 与要分组的轴长度相同的值的列表或数组
- 一个值，指示DataFrame中的列名
- 一个dict或Series，在被分组的轴上的值和组名之间具有对应关系
- 在轴索引或索引中的各个标签上调用的函数

请注意，后三种方法是生成用于拆分对象的值数组的快捷方式。 如果这一切看起来都很抽象，请不要担心。 在本章中，我将给出所有这些方法的许多示例。 首先，这是一个小的表格数据集，作为DataFrame：

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [2]:
df

,key1,key2,data1,data2
0,a,one,1.397355,-1.396320
1,a,two,-0.862800,0.479268
2,b,one,0.315246,-1.148740
3,b,two,0.783404,0.624801
4,a,one,-1.194230,2.645832


假设您想使用key1中的标签来计算data1列的平均值。有很多方法可以做到这一点。一种是访问data1并通过key1的（系列）列调用groupby：

In [6]:
grouped = df['data1'].groupby(df['key1'])

grouped

现在，此 grouped 变量是一个GroupBy对象。除了关于组密钥 `df['key1']` 的一些中间数据外，它实际上还没有计算任何东西。想法是，该对象具有将某些操作应用于每个组所需的所有信息。例如，要计算组均值，我们可以调用GroupBy的mean方法：

In [7]:
grouped.mean()

key1
a   -0.219891
b    0.549325
Name: data1, dtype: float64

稍后，我将详细解释当您调用.mean（）时会发生什么。此处重要的是，已根据组键聚合了数据（系列），从而生成了一个新的系列，该系列现在由key1列中的唯一值索引。

结果索引的名称为'key1'，因为DataFrame列df ['key1']确实如此。

相反，如果我们将多个数组作为列表传递，则会得到不同的结果：

In [13]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [14]:
means

key1  key2
a     one     0.101563
      two    -0.862800
b     one     0.315246
      two     0.783404
Name: data1, dtype: float64

在这里，我们使用两个键对数据进行分组，并且得到的系列现在具有由观察到的唯一键对组成的层次结构索引：

In [17]:
means.unstack()

key2,one,two
key1,,
a,0.101563,-0.862800
b,0.315246,0.783404


在此示例中，组键都是系列，尽管它们可以是长度合适的任何数组：

In [18]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.862800
            2006    0.315246
Ohio        2005    1.090380
            2006   -1.194230
Name: data1, dtype: float64

通常，分组信息与您要处理的数据位于同一DataFrame中。在这种情况下，您可以将列名（无论是字符串，数字还是其他Python对象）传递为组键：

In [19]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.219891,0.57626
b,0.549325,-0.26197


In [21]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.101563  0.624756
     two  -0.862800  0.479268
b    one   0.315246 -1.148740
     two   0.783404  0.624801

在第一种情况下，您可能已经注意到df.groupby（'key1'）.mean（），结果中没有key2列。 因为df ['key2']不是数字数据，所以它被认为是令人讨厌的列，因此将其从结果中排除。 默认情况下，所有数字列都是聚合的，不过您可以很快看到它可以过滤到一个子集。

不管使用groupby的目的如何，通常有用的GroupBy方法是size，它返回一个包含组大小的Series：

In [22]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

请注意，组键中所有缺少的值都将从结果中排除。

### 遍历组

GroupBy对象支持迭代，生成一个2元组的序列，其中包含组名以及数据块。考虑以下：

In [25]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.397355 -1.396320
1    a  two -0.862800  0.479268
4    a  one -1.194230  2.645832
b
  key1 key2     data1     data2
2    b  one  0.315246 -1.148740
3    b  two  0.783404  0.624801


如果有多个键，则元组中的第一个元素将是键值的元组：

In [29]:
for (key1, key2), group in df.groupby(['key1', 'key2']):
    print(key1, key2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  1.397355 -1.396320
4    a  one -1.194230  2.645832
a two
  key1 key2   data1     data2
1    a  two -0.8628  0.479268
b one
  key1 key2     data1    data2
2    b  one  0.315246 -1.14874
b two
  key1 key2     data1     data2
3    b  two  0.783404  0.624801


当然，您可以选择对数据片段进行任何操作。 您可能会发现有用的食谱是将数据片段的字典作为单行计算：

In [30]:
pieces = dict(list(df.groupby('key1')))

In [31]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.315246,-1.148740
3,b,two,0.783404,0.624801


默认情况下，在axis = 0上对groupby groups进行分组，但是您可以在其他任何轴上进行分组。 例如，我们可以在此处按dtype将示例df的列分组，如下所示：

In [36]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [40]:
grouped = df.groupby(df.dtypes, axis=1)

我们可以这样打印组：

In [42]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.397355 -1.396320
1 -0.862800  0.479268
2  0.315246 -1.148740
3  0.783404  0.624801
4 -1.194230  2.645832
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选择列或列子集

使用列名或列名数组对从DataFrame创建的GroupBy对象建立索引会对列子集的聚合产生影响。这意味着：

```python
    df.groupby('key1')['data1']
    df.groupby('key2')['data2']
```

是用于以下方面的语法糖：

```python
    df['data1'].groupby(df['key1'])
    df['data2'].groupby(df['key2'])
```

特别是对于大型数据集，可能希望仅汇总几列。例如，在前面的数据集中，要仅计算data2列的均值并将结果作为DataFrame获得，我们可以编写：

In [50]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.624756
     two   0.479268
b    one  -1.148740
     two   0.624801

如果传递了列表或数组，则此索引操作返回的对象是分组的DataFrame；如果仅将单个列名称作为标量传递，则该分组的数据对象是分组的Series：

In [53]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [54]:
s_grouped.mean()

key1  key2
a     one     0.624756
      two     0.479268
b     one    -1.148740
      two     0.624801
Name: data2, dtype: float64

### 用字典和系列分组

分组信息可能以数组以外的形式存在。让我们考虑另一个示例DataFrame：

In [59]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

In [60]:
people

,a,b,c,d,e
Joe,-0.302145,-1.691325,-1.241065,-0.067045,0.351536
Steve,-0.295473,-0.567282,0.234416,-0.903261,0.663051
Wes,-0.644371,NaN,NaN,1.108819,0.031416
Jim,0.835311,-0.753950,-1.550279,0.219683,0.184482
Travis,0.334462,-0.782174,-0.320201,-0.273930,-0.278756


现在，假设我对各列有一个组对应关系，并希望按组将各列加起来：

In [61]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 
           'd': 'blue', 'e': 'red', 'f': 'orange'}

现在，您可以从此dict构造一个数组以传递给groupby，但是我们可以仅传递dict（我添加了键“ f”以突出显示未使用的分组键是可以的）：

In [62]:
by_column = people.groupby(mapping, axis=1)

In [63]:
by_column.sum()

,blue,red
Joe,-1.308110,-1.641934
Steve,-0.668845,-0.199703
Wes,1.108819,-0.612955
Jim,-1.330595,0.265844
Travis,-0.594131,-0.726468


系列具有相同的功能，可以将其视为固定大小的映射：

In [64]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [65]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 按函数分组

与dict或Series相比，使用Python函数是定义组映射的更通用的方法。 每个索引值都会调用一次作为组键传递的任何函数，并将返回值用作组名。 更具体地说，请考虑上一节中的示例DataFrame，该示例以人们的名字作为索引值。 假设您想按名称的长度分组； 虽然您可以计算一个字符串长度的数组，但只需传递len函数会更简单：

In [66]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.111205,-2.445274,-2.791344,1.261458,0.567434
5,-0.295473,-0.567282,0.234416,-0.903261,0.663051
6,0.334462,-0.782174,-0.320201,-0.273930,-0.278756


将函数与数组，字典或系列混合使用不是问题，因为所有内容都在内部转换为数组：

In [69]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.644371 -1.691325 -1.241065 -0.067045  0.031416
  two  0.835311 -0.753950 -1.550279  0.219683  0.184482
5 one -0.295473 -0.567282  0.234416 -0.903261  0.663051
6 two  0.334462 -0.782174 -0.320201 -0.273930 -0.278756

### 按索引级别分组

分层索引数据集的最后一个便利是使用轴索引级别之一进行聚合的能力。让我们看一个例子：

In [72]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.356800 -0.298948 -0.583277 -0.930820 -0.686415
1      0.699103  0.075397 -0.734981 -1.023878  2.173288
2      0.426617  0.431732 -0.062289 -1.401592 -0.621200
3     -1.378141  1.859125 -0.907661 -0.376833  0.026421

要按级别分组，请使用level关键字传递级别编号或名称：

In [73]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
